In [ ]:
# Import necessary packages
import polars as pl

In [ ]:
# Load the bike trips and weather data
trips = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv", null_values='NA')
weather = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv", null_values=['NA',''])

trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
print(trips.height)
print(weather.height)

78704
733


In [ ]:
import polars as pl

trips_weather = trips.join(weather, on="date", how="inner")

weather_cols = set(trips_weather.columns)

def rainy_expr():
    if "rain" in weather_cols:
        return pl.col("rain").cast(pl.Boolean, strict=False).fill_null(False)

    if "events" in weather_cols:
        return pl.col("events").cast(pl.Utf8, strict=False).str.contains("Rain", literal=True, strict=False).fill_null(False)

    for c in ["precipitation", "precip", "prcp"]:
        if c in weather_cols:
            return pl.col(c).cast(pl.Float64, strict=False).fill_null(0) > 0

    return pl.lit(False)

trips_weather = trips_weather.with_columns(rainy=rainy_expr())

avg_duration_rain = (
    trips_weather
    .filter(pl.col("rainy") == True)
    .select(pl.col("duration").cast(pl.Float64, strict=False))
    .select(pl.col("duration").mean().alias("avg_duration_rain"))
)

avg_duration_rain

avg_duration_rain
f64
10.545472


In [ ]:
# Load baseball datasets
Batting = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv", null_values='NA')
People = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv", null_values='NA')
HallOfFame = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv", null_values='NA')

In [ ]:
import polars as pl

people_cols = set(People.columns)

if "birthState" in people_cols:
    florida_filter = pl.col("birthState").cast(pl.Utf8, strict=False).str.to_uppercase() == "FL"
elif "birth_state" in people_cols:
    florida_filter = pl.col("birth_state").cast(pl.Utf8, strict=False).str.to_uppercase() == "FL"
else:
    alt_state_col = next((c for c in ["state", "stateOfBirth"] if c in people_cols), None)
    florida_filter = pl.col(alt_state_col).cast(pl.Utf8, strict=False).str.to_uppercase().eq("FL") if alt_state_col else pl.lit(False)

batting_people = Batting.join(People, on="playerID", how="inner")

total_hr_florida = (
    batting_people
    .filter(florida_filter)
    .select(pl.col("HR").cast(pl.Int64, strict=False).fill_null(0))
    .select(pl.col("HR").sum().alias("total_HR_players_born_in_FL"))
)

total_hr_florida

total_HR_players_born_in_FL
i64
16225


In [30]:
import polars as pl

hall_inducted = (
    HallOfFame
    .with_columns(
        inducted_txt = pl.col("inducted").cast(pl.Utf8, strict=False).str.to_uppercase()
    )
    .with_columns(
        is_inducted = (
            (pl.col("inducted_txt") == "Y") |
            (pl.col("inducted_txt") == "YES") |
            (pl.col("inducted_txt") == "TRUE") |
            (pl.col("inducted_txt") == "T") |
            (pl.col("inducted_txt") == "1")
        )
    )
    .filter(pl.col("is_inducted"))
    .select("playerID")
    .unique()
)

career_hits = (
    Batting
    .join(hall_inducted, on="playerID", how="inner")
    .group_by("playerID")
    .agg(total_hits = pl.col("H").cast(pl.Int64, strict=False).fill_null(0).sum())
)


avg_career_hits_hof = career_hits.select(
    pl.col("total_hits").mean().alias("avg_career_hits_hall_of_famers")
)

avg_career_hits_hof

avg_career_hits_hall_of_famers
f64
1624.199262
